In [1]:
import torch
cuda_ava = "Ture" if torch.cuda.is_available() else "False"
cuda_device = torch.cuda.get_device_name()
print("pytorch test:"+ cuda_ava +"\nDevice name:"+cuda_device)

pytorch test:Ture
Device name:NVIDIA GeForce GTX 1080 Ti


In [2]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import transformers

In [3]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
#from torchsummary import summary
from tqdm import tqdm

In [4]:
class BertDataset(Dataset):
    def __init__(self, tokenizer,max_length):
        super(BertDataset, self).__init__()
        #self.root_dir=root_dir
        self.train_csv=pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        
    def __len__(self):
        return len(self.train_csv)
    
    def __getitem__(self, index):
        
        text1 = self.train_csv.iloc[index,0]
        
        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long).cuda(),
            'mask': torch.tensor(mask, dtype=torch.long).cuda(),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).cuda(),
            'target': torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long).cuda()
            }
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

dataset= BertDataset(tokenizer, max_length=100)

dataloader=DataLoader(dataset=dataset,batch_size=32)

In [13]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(768,1)
        self.relu =nn.ReLU()
    def forward(self,ids,mask,token_type_ids):
        _,o2 = self.bert_model(ids,attention_mask = mask, token_type_ids= token_type_ids, return_dict=False)
        
        o2= self.dropout(o2)
        lin_out= self.out(o2)

        out = self.relu(lin_out)
        return out

model = BERT()
model = model.cuda()

loss_fn = nn.BCEWithLogitsLoss()
loss_fn = loss_fn.cuda()

optimizer = optim.Adam(model.parameters(),lr=1e-5)

print(model)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [14]:
for param in model.bert_model.parameters():
    param.requires_grad = False

In [11]:
def finetune(epochs,dataloader,model,loss_fn,optimizer):
    model.train()
    for  epoch in range(epochs):
        running_loss = 0.0
        loop = tqdm(enumerate(dataloader),leave=False,total=len(dataloader),mininterval=10)
        for batch, dl in loop:
            
            #data things
            ids=dl['ids']
            token_type_ids=dl['token_type_ids']
            mask= dl['mask']
            label=dl['target']
            label = label.unsqueeze(1)
            
            #from cpu to GPU
            ids= ids.cuda()
            token_type_ids= token_type_ids.cuda()
            mask= mask.cuda()
            label= label.cuda()
            
            #model update (梯度歸0)
            optimizer.zero_grad()
            
            #
            output=model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids)
            label = label.type_as(output)

            loss=loss_fn(output,label)
            #backward
            loss.backward()
            
            
            optimizer.step()
            
            running_loss += loss.item()
            
            output = output.cpu()
            pred = np.where(output >= 0, 1, 0)

        
            num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
            num_samples = pred.shape[0]
            
            
            accuracy = num_correct/num_samples
            
            print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
            
            # Show progress while training
            loop.set_description(f'Epoch={epoch}/{epochs}')
            loop.set_postfix(loss=loss.item(),acc=accuracy)
    
    return model

In [15]:
epoch =6
model=finetune(epoch, dataloader, model, loss_fn, optimizer)

Epoch=0/6:   0%|          | 0/217 [00:00<?, ?it/s, acc=0.75, loss=0.67]  

Got 15 / 32 with accuracy 46.88
Got 24 / 32 with accuracy 75.00


Epoch=0/6:   0%|          | 0/217 [00:00<?, ?it/s, acc=0.562, loss=0.676]

Got 18 / 32 with accuracy 56.25
Got 18 / 32 with accuracy 56.25


Epoch=0/6:   0%|          | 0/217 [00:00<?, ?it/s, acc=0.531, loss=0.688]

...

Epoch=2/6:  37%|███▋      | 81/217 [00:18<00:16,  8.09it/s, acc=0.5, loss=0.7]    

Got 14 / 32 with accuracy 43.75
Got 16 / 32 with accuracy 50.00


Epoch=2/6:  37%|███▋      | 81/217 [00:18<00:16,  8.09it/s, acc=0.406, loss=0.697]

Got 19 / 32 with accuracy 59.38
Got 13 / 32 with accuracy 40.62


KeyboardInterrupt: 